# Imports

In [1]:
import os 
import sys
import pandas as pd
import psycopg2 
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0,parent_dir)

In [20]:
from importlib import reload
import scripts.data_loader, scripts.clean_and_transform, scripts.analysis_1, scripts.visualization, scripts.analysis_2, scripts.analysis_3
reload(scripts.data_loader)
reload(scripts.clean_and_transform)
reload(scripts.analysis_1)
reload(scripts.analysis_2)
reload(scripts.analysis_3)
reload(scripts.visualization)

<module 'scripts.visualization' from 'c:\\ML and DS Files\\Kifiya AI\\Kaim-week-2\\scripts\\visualization.py'>

In [4]:
from scripts.data_loader import DataLoader, LoadSqlData
from scripts.clean_and_transform import DropNullRows, NullValueFiller, DropUndefined
from scripts.analysis_1 import UserSessionAggregator, DataDescriber, VariableTransformer, MetricsAnalyzer, DispersionAnalyzer, PCAAnalyzer, HandsetAnalysis
from scripts.analysis_2 import TelecomEngagementAnalysis, TelecomEngagementAnalysis_2
from scripts.analysis_3 import TelecomAnalysis
from scripts.visualization import UnivariateAnalyzer, BivariateAnalyzer, CorrelationAnalyzer

# Load data from postgreSQL

In [10]:
# Define your SQL query
query = "SELECT * FROM xdr_data"
# Create an instance of the LoadSqlData class
data_loader = LoadSqlData(query)
# Load data using psycopg2
data= data_loader.load_data_using_sqlalchemy()
data.head()

Sucessfully Loaded


,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


# Clean the data

In [11]:
data.shape

(150001, 55)

In [12]:
# Drop  undefined values from Handset Type
drop_undefined = DropUndefined(data)
data = drop_undefined.DeleteUndefined(column='Handset Type', value='undefined')

Sucessfully droped undefined columns


In [13]:
# drop null rows for the follwing columns
# Bearer Id, Start, End, IMSI, MSISDN/Number, IMEI,Last Location Name, Handset Manufacturer, Handset Type
col_1 = ['Bearer Id', 'Start', 'End', 'IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name', 'Handset Manufacturer', 'Handset Type']
dropper = DropNullRows(columns_to_check=col_1)

# Drop rows where the specified column has null values
data = dropper.drop_if_null(data)

Sucessfuly dropped null rows from ['Bearer Id', 'Start', 'End', 'IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name', 'Handset Manufacturer', 'Handset Type']


In [14]:
null_columns = ['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 
           'DL TP < 50 Kbps (%)','50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 
           'UL TP < 10 Kbps (%)','10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
            'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B',
            'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 'Nb of sec with 6250B < Vol DL < 31250B',
            'Nb of sec with 6250B < Vol UL < 37500B','Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B']


In [ ]:
# Numerical null values are filled based on the outlier and normal distribution
# Initialize the NullValueFiller class
filler = NullValueFiller(data, null_columns)
    
# Fill null values based on mean/median decision
filler.fill_nulls()

In [19]:
null_counts = data.isnull().sum()
if null_counts.sum() > 0:
    print('Null value present please check the dataframe')
else:
    print('All columns are not null')

All columns are not null


# Task 3

In [ ]:
# Main script for analysis
if __name__ == "__main__":
    file_path = 'telecom_data.csv'  # Replace with the actual dataset path
    telecom_analysis = TelecomAnalysis(file_path)

    # Preprocess data
    telecom_analysis.preprocess_data()

    # Task 1: Aggregate metrics per customer
    aggregated_data = telecom_analysis.aggregate_per_customer()
    print("Aggregated Data:\n", aggregated_data.head())

    # Task 2: Compute top, bottom, and most frequent for metrics
    for metric in ['TCP Retransmission', 'RTT', 'Throughput']:
        top_10, bottom_10, frequent = telecom_analysis.compute_top_bottom_frequent(metric)
        print(f"Top 10 {metric}: {top_10}")
        print(f"Bottom 10 {metric}: {bottom_10}")
        print(f"Most Frequent {metric}: {frequent}")

    # Task 3: Distribution analysis
    throughput_distribution = telecom_analysis.distribution_analysis('Throughput', 'Handset Type')
    print("Throughput Distribution:\n", throughput_distribution.head())

    tcp_distribution = telecom_analysis.distribution_analysis('TCP Retransmission', 'Handset Type')
    print("TCP Retransmission Distribution:\n", tcp_distribution.head())

    # Task 4: K-means clustering
    features = ['TCP Retransmission', 'RTT', 'Throughput']
    cluster_data, cluster_centers = telecom_analysis.kmeans_clustering(features)
    print("Cluster Data:\n", cluster_data.head())
    print("Cluster Centers:\n", cluster_centers)

    # Visualize clusters
    plt.scatter(telecom_analysis.data['TCP Retransmission'], telecom_analysis.data['Throughput'], c=telecom_analysis.data['Cluster'])
    plt.title("Clusters Visualization")
    plt.xlabel("TCP Retransmission")
    plt.ylabel("Throughput")
    plt.show()